In [1]:
import tensorflow.compat.v1 as tf

In [2]:
import numpy as np
#import tensorflow as tf

LARGE_NUM = 1e9
temperature = 1.0
weights = 1.0

In [3]:
z =np.array([[1.0, 2.0],
        [3.0, -2.0],
        [1.0, 5.0],
        [1.0, 0.75],
        [2.8,-1.75],
        [1.0, 4.7]])

In [4]:
z

array([[ 1.  ,  2.  ],
       [ 3.  , -2.  ],
       [ 1.  ,  5.  ],
       [ 1.  ,  0.75],
       [ 2.8 , -1.75],
       [ 1.  ,  4.7 ]])

In [5]:
z.shape

(6, 2)

In [6]:
hidden = tf.math.l2_normalize(z, -1)

In [7]:
hidden

<tf.Tensor: shape=(6, 2), dtype=float64, numpy=
array([[ 0.4472136 ,  0.89442719],
       [ 0.83205029, -0.5547002 ],
       [ 0.19611614,  0.98058068],
       [ 0.8       ,  0.6       ],
       [ 0.8479983 , -0.52999894],
       [ 0.20810764,  0.97810593]])>

In [8]:
hidden1, hidden2 = tf.split(hidden, 2, 0)

In [9]:
hidden1

<tf.Tensor: shape=(3, 2), dtype=float64, numpy=
array([[ 0.4472136 ,  0.89442719],
       [ 0.83205029, -0.5547002 ],
       [ 0.19611614,  0.98058068]])>

In [10]:
hidden2

<tf.Tensor: shape=(3, 2), dtype=float64, numpy=
array([[ 0.8       ,  0.6       ],
       [ 0.8479983 , -0.52999894],
       [ 0.20810764,  0.97810593]])>

In [11]:
batch_size = tf.shape(hidden1)[0]

In [12]:
batch_size

<tf.Tensor: shape=(), dtype=int32, numpy=3>

In [13]:
hidden1_large = hidden1

In [14]:
hidden2_large = hidden2

In [15]:
labels = tf.one_hot(tf.range(batch_size), batch_size * 2,  dtype='float64')

In [16]:
labels

<tf.Tensor: shape=(3, 6), dtype=float64, numpy=
array([[1., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0.]])>

In [17]:
masks = tf.one_hot(tf.range(batch_size), batch_size, dtype='float64')

In [18]:
masks

<tf.Tensor: shape=(3, 3), dtype=float64, numpy=
array([[1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.]])>

In [19]:
logits_aa = tf.matmul(hidden1, hidden1_large, transpose_b=True) / temperature

In [20]:
logits_aa

<tf.Tensor: shape=(3, 3), dtype=float64, numpy=
array([[ 1.        , -0.12403473,  0.96476382],
       [-0.12403473,  1.        , -0.38074981],
       [ 0.96476382, -0.38074981,  1.        ]])>

In [21]:
logits_aa = logits_aa - masks * LARGE_NUM

In [22]:
logits_aa

<tf.Tensor: shape=(3, 3), dtype=float64, numpy=
array([[-9.99999999e+08, -1.24034735e-01,  9.64763821e-01],
       [-1.24034735e-01, -9.99999999e+08, -3.80749805e-01],
       [ 9.64763821e-01, -3.80749805e-01, -9.99999999e+08]])>

In [23]:
logits_bb = tf.matmul(hidden2, hidden2_large, transpose_b=True) / temperature
logits_bb = logits_bb - masks * LARGE_NUM

In [24]:
logits_bb

<tf.Tensor: shape=(3, 3), dtype=float64, numpy=
array([[-9.99999999e+08,  3.60399279e-01,  7.53349673e-01],
       [ 3.60399279e-01, -9.99999999e+08, -3.41920176e-01],
       [ 7.53349673e-01, -3.41920176e-01, -9.99999999e+08]])>

In [25]:
logits_ab = tf.matmul(hidden1, hidden2_large, transpose_b=True) / temperature

In [26]:
logits_ab

<tf.Tensor: shape=(3, 3), dtype=float64, numpy=
array([[ 0.89442719, -0.09480909,  0.96791311],
       [ 0.33282012,  0.99956775, -0.36939952],
       [ 0.74524131, -0.35340057,  0.99992504]])>

In [27]:
logits_ba = tf.matmul(hidden2, hidden1_large, transpose_b=True) / temperature

In [28]:
logits_ba

<tf.Tensor: shape=(3, 3), dtype=float64, numpy=
array([[ 0.89442719,  0.33282012,  0.74524131],
       [-0.09480909,  0.99956775, -0.35340057],
       [ 0.96791311, -0.36939952,  0.99992504]])>

In [29]:
loss_a = tf.losses.softmax_cross_entropy(
      labels, tf.concat([logits_ab, logits_aa], 1), weights=weights)

In [30]:
loss_a

<tf.Tensor: shape=(), dtype=float64, numpy=1.1290570497512817>

In [31]:
loss_b = tf.losses.softmax_cross_entropy(
      labels, tf.concat([logits_ba, logits_bb], 1), weights=weights)

In [32]:
loss_b

<tf.Tensor: shape=(), dtype=float64, numpy=1.1363438367843628>

In [33]:
loss = loss_a + loss_b

In [34]:
loss

<tf.Tensor: shape=(), dtype=float64, numpy=2.2654008865356445>

In [35]:
tf.concat([logits_ab, logits_aa], 1)

<tf.Tensor: shape=(3, 6), dtype=float64, numpy=
array([[ 8.94427191e-01, -9.48090926e-02,  9.67913106e-01,
        -9.99999999e+08, -1.24034735e-01,  9.64763821e-01],
       [ 3.32820118e-01,  9.99567754e-01, -3.69399524e-01,
        -1.24034735e-01, -9.99999999e+08, -3.80749805e-01],
       [ 7.45241314e-01, -3.53400569e-01,  9.99925040e-01,
         9.64763821e-01, -3.80749805e-01, -9.99999999e+08]])>

In [36]:
tf.nn.softmax_cross_entropy_with_logits_v2(labels=labels, logits=tf.concat([logits_ab, logits_aa], 1))

<tf.Tensor: shape=(3,), dtype=float64, numpy=array([1.3563848 , 0.85200828, 1.178778  ])>

In [37]:
np.sum(tf.nn.softmax_cross_entropy_with_logits_v2(labels=labels, logits=tf.concat([logits_ab, logits_aa], 1)).numpy())/3

1.1290570250817762